# Prediction check for manual cascading

This notebook is derived from prediction_virtual_categories_138_1_vs_all_model.ipynb.

## 1. Set up

In [ ]:
import os
import json
import glob
import scipy
import numpy as np

# import preprocess func from processor module
from processor import DataSet

from keras.models import model_from_json
from keras.models import model_from_yaml
from keras.utils import np_utils
from keras.preprocessing import image

import pandas as pd
import glob

In [ ]:
# set root as ../
import sys
sys.path.append("../")

In [ ]:
# created by virtual_categories.ipynb
CATSTORE_PATH = ""

In [ ]:
BASE_MODEL_DIR = "./trained_model/inceptionv3/"
BASE_NAME = ""
CODE_EXTRACTOR=""
BASE_CODE_NAME = "code_" + BASE_NAME

In [ ]:
from one_vs_all import ModelBinder
from codeextractor import DNNCodeExtractor

def ModelBinder_create(base_model_name = BASE_CODE_NAME, basedir = BASE_MODEL_DIR, extractor_path=CODE_EXTRACTOR):
    return ModelBinder(base_model_name, basedir, DNNCodeExtractor.create_from(extractor_path))

In [ ]:
from one_vs_all import ManualCascader

In [ ]:
CHECK_DATA_DIR1 = ''
CHECK_DATA_DIR2 = ''

In [ ]:
CHECK_DATA_PATHS1 = list(glob.glob(os.path.normpath("{}/*.jpg").format(CHECK_DATA_DIR1)))
CHECK_DATA_PATHS2 = list(glob.glob(os.path.normpath("{}/*.jpg").format(CHECK_DATA_DIR2)))

In [ ]:
import random, itertools

In [ ]:
# CHECK_DATA_PATHS = list(itertools.chain(random.sample(CHECK_DATA_PATHS1, 5000), CHECK_DATA_PATHS2))
CHECK_DATA_PATHS = list(itertools.chain(CHECK_DATA_PATHS1[0:5000], CHECK_DATA_PATHS2[0:5000]))

In [ ]:
len(CHECK_DATA_PATHS)

## Second category trial

In [ ]:
CHECKDF_PATH=""

In [ ]:
from category import VirtualCategories
vc = VirtualCategories.from_file(CATSTORE_PATH)
catkeys = vc.keys()

In [ ]:
df = pd.read_pickle(CHECKDF_PATH)

In [ ]:
# for 108_72
keyset = set(['10', '105', '108_72', '109', '111', '137_170', '147', '151', '156_79','162', '17', '18', '20', '21', '39', '4', '40', '49', '5', '57', '8','83', '92', '93'])
h1key = "108_72"

In [ ]:
# keyset = set(['108_72', '109', '93', '17', '18'])
# h1key = '93'

### There seems no argmax=='93' record. Use some threshold instead.

In [ ]:
filtered = df[df['93'] > 0.89]

### Use 0.96 for 108_72

In [ ]:
df.sort_values(by=h1key, ascending = False)[h1key][20:40]

In [ ]:
#plot_image_list(df.sort_values(by=h1key, ascending = False)['filepaths'][40:60].values)

In [ ]:
filtered = df[df[h1key] > 0.96]

In [ ]:
extractor = DNNCodeExtractor.create_from(CODE_EXTRACTOR)

In [ ]:
from models.modelutils import load_best_model

In [ ]:
def second_filter(filtered, targetkey):
    files = filtered["filepaths"].values
    bottlnecks= np.array(list(extractor.map_code(files)))
    model = load_best_model(os.path.join(BASE_MODEL_DIR, "{}_{}".format(BASE_CODE_NAME, "h1_" + targetkey)))
    res = model.predict(bottlnecks)
    df = pd.DataFrame({targetkey:res[:,1], 'filepaths':files})
    return df.sort_values(by=targetkey, ascending=False)

In [ ]:
secdf = second_filter(filtered, h1key)

In [ ]:
secdffiltered = secdf[secdf[h1key]>0.5]

In [ ]:
from visualize import plot_image_list

In [ ]:
def plot_top20(df, targetkey):
    scores = df[targetkey]

    print(", ".join(str(val) for val in scores[0:5]))
    print(", ".join(str(val) for val in scores[5:10]))
    print(", ".join(str(val) for val in scores[10:15]))
    print(", ".join(str(val) for val in scores[15:20]))
    plot_image_list(df['filepaths'].values[0:20])

In [ ]:
# plot_top20(secdffiltered, '108_72')

In [ ]:
# plot_top20(secdffiltered, '93')

## Second category trial2

In [ ]:
h2pasta = df[df["argmax"].isin(keyset)]

In [ ]:
resdf = second_filter(h2pasta, '93')

In [ ]:
res = resdf[resdf['93']> 0.5]

In [ ]:
# plot_top20(res, '93')

In [ ]:
# why gyudon?

file = res[res['93'] == 0.897757]['filepaths'].values[0]

In [ ]:
h2pasta[h2pasta['filepaths'] == file]

In [ ]:
h2pasta[h2pasta['filepaths'] == file]['17']

# Manual Cascading predict

In [ ]:
ens = ModelBinder_create()

In [ ]:
# H2 classifier test
key_thrds_pair = [
('109',  0.855) #pasta, confused to yakisoba
,('108_72', 0.961) # yakisoba, confused to pasta, 
,('17', 0.99)
,('18', 0.97)
,('93', 0.905)
]

In [ ]:
# with ramen
key_thrds_pair = [
('109',  0.855) #pasta, confused to yakisoba
,('108_72', 0.961) # yakisoba, confused to pasta, 
,('17', 0.99)
,('18', 0.97)
,('93', 0.905)
,('39', 0.85)
]

In [ ]:
manuens= ManualCascader(key_thrds_pair, ens)

In [ ]:
manuens.load_all_models()

# Normal predict version

In [ ]:
CHUNKSIZE=3000
chunks = [CHECK_DATA_PATHS[x:x + CHUNKSIZE] for x in range(0, len(CHECK_DATA_PATHS), CHUNKSIZE)]

In [ ]:
from tqdm import tqdm
import itertools

In [ ]:
res = [manuens.predict(chunk) for chunk in tqdm(chunks)]

In [ ]:
# flatten result
res = list(itertools.chain.from_iterable(res))

In [ ]:
len(res)

In [ ]:
respair = list(zip(res, CHECK_DATA_PATHS))

In [ ]:
len(respair)

In [ ]:
OTHER = manuens.otherclass

In [ ]:
resdf = pd.DataFrame(respair)

In [ ]:
resdf.columns = ["category", "filepath"]

In [ ]:
DF_PATH="trained_model/intermediate/flickr_h2_with_ramen.dat"

In [ ]:
resdf.to_pickle(DF_PATH)

In [ ]:
# load from file
resdf = pd.read_pickle(DF_PATH)

### df evaluation

In [ ]:
resdf.count()['category']

In [ ]:
resdf.where(resdf["category"] != OTHER).sort_values(by="category").dropna().count()['category']

In [ ]:
from category import VirtualCategories
import os
from visualize import plot_image_list

In [ ]:
vc = VirtualCategories.from_file(CATSTORE_PATH)

In [ ]:
def plot_category(classidx):
    print(vc.name(manuens.class2catname(classidx)))
    plot_image_list(resdf.where(resdf["category"] == classidx).dropna()["filepath"].values)


## Sub category trial

In [ ]:
keyset = set(['108_72', '109', '93', '17', '18', '39'])
h1keyset = set(['93', '39'])


#keyset = set(['108_72', '109', '93', '17', '18'])
## 17, 18, 93 is not included
#keyset = set(['108_72', '109'])

In [ ]:
keytoidx = {manuens.class2catname(idx):idx for idx in range(OTHER)}

In [ ]:
idxset = {keytoidx[key] for key in keyset}

In [ ]:
idxset

In [ ]:
h2df = resdf[resdf["category"].isin(idxset)].dropna()

In [ ]:
h2files = h2df["filepath"].values

In [ ]:
extractor = ens.extractor

In [ ]:
bottlnecks= np.array(list(extractor.map_code(h2files)))

In [ ]:
from models.modelutils import load_best_model

In [ ]:
def predict_score(key):
    model = load_best_model(os.path.join(BASE_MODEL_DIR, "{}_{}".format(BASE_CODE_NAME, "h1_" + key)))
    res = model.predict(bottlnecks)
    return res[:, 1]

In [ ]:
scoredict = {key:predict_score(key) for key in h1keyset}

In [ ]:
df = pd.DataFrame(scoredict)

In [ ]:
df['filepaths'] = h2files

In [ ]:
df

In [ ]:
scores = res[:, 1]

In [ ]:
len(scores)

In [ ]:
len(h2files)

In [ ]:
df = pd.DataFrame({'h1_109':scores, 'filepaths':h2files})

In [ ]:
df[0:5]

In [ ]:
df.sort_values(by="109", ascending= False)[0:20]

In [ ]:
from visualize import plot_image_list
def plot_target_top20_paging(target_key, start):
    targetsorted = df.sort_values(by=target_key, ascending=False)[start:(start+20)]
    print(vc.name(target_key))
    maxes = targetsorted[target_key].values
    print(", ".join(str(val) for val in maxes[0:5]))
    print(", ".join(str(val) for val in maxes[5:10]))
    print(", ".join(str(val) for val in maxes[10:15]))
    print(", ".join(str(val) for val in maxes[15:]))
    plot_image_list(targetsorted['filepaths'].values)

In [ ]:
def plot_target_top20_2(target_key):
    plot_target_top20_paging(target_key, 0)

In [ ]:
def plot_target_sectop20_2(target_key):
    plot_target_top20_paging(target_key, 20)

In [ ]:
# plot_target_top20_2('93')

In [ ]:
# plot_target_sectop20_2('93')

In [ ]:
plot_target_top20_paging('93', 40)

In [ ]:
# plot_target_top20_2('39')

In [ ]:
# plot_target_sectop20_2('39')

In [ ]:
# plot_target_top20_2('109')

In [ ]:
# plot_target_top20_2('108_72')

In [ ]:
# plot_target_top20_2('17')

In [ ]:
# plot_target_top20_2('18')

In [ ]:
# plot_target_top20_2('93')

## Collect h2 files

In [ ]:
h2df = resdf.where(resdf["category"] != OTHER).sort_values(by="category").dropna()

In [ ]:
files = h2df["filepath"].values

In [ ]:
import glob
import shutil
H2 = 'data/h2'

In [ ]:
def move_to_broken(f):
    moveddir = os.path.dirname(os.path.join(H2, f))
    os.path.isdir(moveddir) or os.makedirs(moveddir)
    shutil.copy(f, moveddir)
    os.remove(f)
    print("{} is broken, move it.".format(f))

In [ ]:
os.path.basename(files[0])

In [ ]:

os.makedirs(H2, exist_ok=True)

for f in files:
    movepath = os.path.join(H2, f)
    movedir = os.path.dirname(movepath)
    os.makedirs(movedir, exist_ok = True)
    
    os.link(f, movepath)
